In [14]:
from dotenv import load_dotenv
import os
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [15]:
import sqlite3

conn = sqlite3.connect('test.db')
c = conn.cursor()
            
c.execute('''
          CREATE TABLE IF NOT EXISTS products
          (
            [id] INTEGER PRIMARY KEY, 
            [name] TEXT, 
            [supplier] TEXT, 
            [store] TEXT
          )
          ''')

c.execute('''
          CREATE TABLE IF NOT EXISTS prices
          (
              [id] INTEGER PRIMARY KEY, 
              [price] INTEGER,
              [discount_pcnt] INTEGER
          )
          ''')   
            
  # Delete existing records
  
c.execute('DELETE FROM products')
c.execute('DELETE FROM prices')

In [16]:
# Data to be inserted into products table
products_data = [
    (1, 'Iphone', 'Apple', 'Nairobi'),
    (2, 'MacBook', 'Apple', 'Mombasa'),
    (3, 'Tablet', 'Samsung', 'Nairobi'),
    (4, 'Desk', 'DELL', 'New York'),
    (5, 'Ipod', 'Apple', 'New Delhi')
]

# Data to be inserted into prices table
prices_data = [
    (1, 1000, 5),
    (2, 3000, 7),
    (3, 2300, 10),
    (4, 1500, 12),
    (5, 3500, 3)
]

# Insert data into products table
c.executemany('INSERT INTO products VALUES (?, ?, ?, ?)', products_data)

# Insert data into prices table
c.executemany('INSERT INTO prices VALUES (?, ?, ?)', prices_data)

# Commit the transaction
conn.commit()

# Close the connection
conn.close()

print("Data has been inserted successfully.")

Data has been inserted successfully.


In [22]:
from langchain import LLMMathChain, OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain_experimental.sql import SQLDatabaseChain
from langchain_community.llms import OpenAI
from langchain.agents import AgentType
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI
from langchain_community.utilities.sql_database import SQLDatabase
import groq
from tavily import TavilyClient


db = SQLDatabase.from_uri("sqlite:///test.db")

# Create LLM model
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125", openai_api_key=OPENAI_API_KEY)

# Create an LLM math tool
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

# Create the database chain
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)


api_key = "YOUR_API_KEY"
tavily = TavilyClient(api_key=TAVILY_API_KEY)

# Define a function to use Tavily for searches
def tavily_search(query):
    response = tavily.search(query=query)
    # Print the response to inspect its structure
    print("Tavily response:", response)
    if 'results' in response:
        return response['results'][0]['content']
    else:
        return "No relevant results found."

# Create tools
tools = [
    Tool(
        name="SearchTool",
        func=tavily_search,
        description="Useful for when you need to answer questions about current events. You should ask targeted questions."
    ),
    Tool(
        name="MathTool",
        func=llm_math_chain.run,
        description="Useful for when you need to answer questions about math."
    ),
    Tool(
        name="Product_Database",
        func=db_chain.run,
        description="Useful for when you need to answer questions about products."
    )
]

# Creating the agent
agent = initialize_agent(
    tools=tools, llm=llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)

# Ask the LLM a question
result = agent.run("Hvilket produkt er billigst fra test databasen? Og Hvilket merke er det?")
print(result)



> Entering new AgentExecutor chain...

Invoking: `Product_Database` with `cheapest`




> Entering new SQLDatabaseChain chain...
cheapest
SQLQuery:SELECT "name", "price" 
FROM products 
JOIN prices ON products.id = prices.id 
ORDER BY "price" ASC 
LIMIT 1;
SQLResult: [('Iphone', 1000)]
Answer:Iphone at a price of 1000 is the cheapest product.
> Finished chain.
Iphone at a price of 1000 is the cheapest product.
Invoking: `Product_Database` with `brand of cheapest product`




> Entering new SQLDatabaseChain chain...
brand of cheapest product
SQLQuery:SELECT "name", MIN("price") AS min_price
FROM products
JOIN prices ON products.id = prices.id
ORDER BY min_price
LIMIT 1;
SQLResult: [('Iphone', 1000)]
Answer:The brand of the cheapest product is Iphone.
> Finished chain.
The brand of the cheapest product is Iphone.Det billigste produktet i testdatabasen er en iPhone til en pris på 1000. Merket på det billigste produktet er også iPhone.

> Finished chain.
Det billigste produktet i testdat